# 認識歷史回測

當我們在市場上交易時，若是對當前市場的趨勢變動沒有把握時，就需要歷史回測(Back Testing)模擬真實的開盤狀況，借此來驗證自己的交易策略是否是有效的。回測不僅僅只是數學計量模型的計算，也包含了計量模型以外的市場分析。

以往大家都沒有完整的歷史數據、只有零散在各處的新聞媒體或券商的統計、盤後資料時，是很難進行歷史回測的。而現在由於資料取得容易，計量回測就成了一個不可或缺的工具。

接下來我們將學會一個技術指標，並且利用我們之前學會的 Python 技能來計算該指標，最後再根據該指標對臺積電的歷史資料進行回測

# 先 import 我們所需要的模組

```python
import xlwings as xw

# 打開你的 tsmc_back_test.xlsx 檔案
wb = xw.Book(r"範例檔案 tsmc_back_test.xlsx 的路徑")
tsmc_sheet = wb.sheets["2330"]
```

In [1]:
import xlwings as xw

# 打開你的 tsmc_back_test.xlsx 檔案
wb = xw.Book(r"tsmc_back_test.xlsx")
tsmc_sheet = wb.sheets["2330"]

## 計算加權移動平均(WMA)

- 簡單移動平均給予所有歷史資料相同的權重
- 但相對來說越近期的資料其實與未來價格越相關，應改與其較高的影響力
- 因此加權移動平均給予越近期的資料越大的權重，用以改善簡單移動平均

![](https://drive.google.com/uc?export=download&id=17rgOsqBKLggR-dCySXh-KTX5sj4kOrQd)

計算方法如下：

- 我們總共有三天，權重分為：3、2、1。
- 因此，計算平均的分母為：3+2+1=6。
- 12/11 之三日加權移動平均為：

![](https://drive.google.com/uc?export=download&id=1O4Q4SBzDzDr5zOG1-3M-RzLMqwqVVaxw)


## 動態截取資料的筆數

In [4]:
# 會 VBA 的人，馬上可以看出 xlwings 原汁原味的實作了 Python 版的 End() 函數
from xlwings.constants import Direction

# 從 B1 儲存格開始，往下查找到最後一個有值的儲存格
last_cell = tsmc_sheet.range("A1").end(Direction.xlDown)
# 把從 A1 開始，最右下角的儲存格底色換成紅色
last_row = last_cell.row
last_row
# 這樣我們就不會因爲試算表資料的改變而需要頻繁的修改程式碼

193

接下來我們用設定 Excel 公式的方法計算第一筆 5 日加權移動平均：

```python
tsmc_sheet.range("C6").formula = "=SUM(B2:B6*$K$2:$K$6)/SUM($K$2:$K$6)"
```

在使用 Excel 時，記得若要讓下拉的時候，都能讓每一行參照到同樣的範圍(在這個範例是 K2:K6)，一般我們都是透過加上`$`。因為是陣列運算所以要按`ctrl+shift+enter`:

```python
"=SUM(B2:B6*$K$2:$K$6)/SUM($K$2:$K$6)"
```
或是用
```python
"=SUMPRODUCT(B2:B6,weight5d)/SUM(weight5d)"
```

但是這種寫法的壞處是，Excel 公式會變得冗長，大幅降低了可讀性與可維護性，這時，我們可以利用 Excel 的**定義名稱** 功能：

![](https://drive.google.com/uc?export=download&id=1jIReb9HOZYcJTyMalZoZRYr0qABVRhPE)

換句話説，這個功能就是**幫一個儲存格或是範圍取一個綽號**，所以接下來在撰寫公式時，輸入該綽號，就可以參照到固定的儲存格/範圍了：

![](https://drive.google.com/uc?export=download&id=1Jp6eSBdaR1ewMoJLMtnRaaYtRxGZN0p4)

## 動態設定 Excel 儲存格或範圍的名稱

```python
# 可以用 name 屬性定義一個儲存格的名稱
tsmc_sheet.cells(1, "A").name = "hello"
# 同樣的，range 也有 name 屬性
tsmc_sheet.range("A1:A3").name = "world"
```

## 設定我們的範例試算表上的名稱

```python
tsmc_sheet.range("K2:K6").name = "weight5d"
tsmc_sheet.range("K2:K11").name = "weight10d"
```

## 整理一下我們所學的東西...

我們的程式碼就會變得相對簡潔：

```python
for row in range(6, last_row+1):
    formula = f"=SUM(B{row-4}:B{row}*weight5d)/SUM(weight5d)"
    tsmc_sheet.range(f"C{row}").formula_array = formula
```

## 接下來我們算出短期 (5日) 與長期 (10日) 的移動平均

In [0]:
# 5日加權移動平均計算
for i in range(6, last_row+1):
    # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
    formula = f"=SUM(B{i-4}:B{i}*weight5d)/SUM(weight5d)"
    # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
    tsmc_sheet.range(f"C{i}").formula_array = formula

In [0]:
# 10日加權移動平均計算
for i in range(11, last_row+1):
    # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
    formula = f"=SUM(B{i-9}:B{i}*weight10d)/SUM(weight10d)"
    # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
    tsmc_sheet.range(f"D{i}").formula_array = formula

## 流程控制

![flow control](https://drive.google.com/uc?export=download&id=1Zj9Gfgc1QGkdzIM7ED-xKXiCX2BcOz3f)

當我們遇上程式需要針對不同選項執行不同結果時，我們需要學會用程式語言表達 Yes 和 No，以及邏輯上的分支點

## Boolean (布林值)
布林值和整數一樣為資料型別，只有兩個值：

```python
True # 代表成立
False # 代表不成立
```

---
## 比較運算子

| 運算子 | 功能     |
| :----: | :------: |
| ==     | 等於     |
| !=     | 不等於   |
| <      | 小於     |
| >      | 大於     |
| <=     | 小於等於 |
| >=     | 大於等於 |

---
## 範例

```python
'hello' == 'hello'
# True
47 > 47
# False
47 >= 47
# True
```
# if 陳述句

若 `if` 後面的比較式條件成立，也就是結果為 `True`，則執行子句中的程式碼：

```python
num = 5

if num > 5:
    print("num is bigger than 5")
```

# if...else 陳述句

若 `if` 關鍵字後面的比較式結果為 `False`，則忽略 `if` 子句，執行 `else` 的子句：

```python
num = 5

if num > 5:
    print("num is bigger than 5")
else:
    print("num is smaller than or equal to 5")
```

# elif 陳述句

白話文就是指 "否則如果"

當邏輯判斷的可能性 / 選項超出兩種時，可用 `elif`：

```python
num = 5

if num > 5:
    print("num is greater than 5")
elif num < 5:
    print("num is less than 5")
else:
    print("num is equal to 5")
```

## 邏輯運算子
當你需要把多個比較式的結果串連起來時

像是條件一與條件二都必須成立:
```python
if 條件一 == True 與 條件二 == True:
    print("條件一與條件二都成立！")
```

要如何用 Python 程式碼表達這件事？


## 邏輯運算子
當你需要把多個比較式連接起來時

| 運算子 | 功能          | 實例    | 結果  |
| :----: | :-----------: | :-----: | :---: |
| and      | 而且        | True and True | True |
| or       | 或許        | True or False | True |
| not      | 反值        | not True | False |

## and truth table(真值表)

| A | B | A and B |
| :----: | :-----------: | :---: |
| True      | True    |  True     |
| True      | False   | False     |
| False     | True    | False     |
| False     | False   | False     |

## or truth table(真值表)

| A | B | A or B |
| :----: | :-----------: | :---: |
| True      | True    |  True     |
| True      | False   | True     |
| False     | True    | True    |
| False     | False   | False     |

# 我們的交易策略

我們用加權移動平均分別算出一個長期與短期的量化指標，再利用這兩個指標判斷進出場的時機：

- 如果 :五日均價 > 十日均價 = **買入一張**
- 如果 :五日均價 < 十日均價 = **賣出一張**

***備注：非投資建議，本課程提供的資料及交易策略，只可作為參考用途，學員在投資前，務請運用個人獨立思考做出抉擇，如因此招致任何損失，概與本課程無涉。**


## 我們先透過此策略，手動計算第一筆資料：

| 日期        | 收盤價    | 5日加權 | 10日加權 |買入股數 |賣出股數|持有股數|持有資金|總資產|
| ----------- |:--------:|:-------:|:-------:|:------:|:-----:|:------:|:-----:|-----:|
| 2017/10/20  | 237.5    | 237.6667| 236.590 |1000    |0      |1000    |762500 |1000000|

In [2]:
# 計算第一天的交易 2017/10/20
tsmc_sheet.range("E11").value = 1000
tsmc_sheet.range("F11").value = 0
tsmc_sheet.range("G11").value = 1000
tsmc_sheet.range("H11").value = tsmc_sheet.range("L18").value - tsmc_sheet.range("B11").value * 1000
tsmc_sheet.range("I11").value = tsmc_sheet.range("H11").value +  tsmc_sheet.range("B11").value * tsmc_sheet.range("G11").value

In [5]:
# 實作交易策略
for i in range(12, last_row+1):
    # 截取當天的 5日加權移動平均
    short_term_ma = tsmc_sheet.range(f"C{i}").value
    # 截取當天的 10日加權移動平均
    long_term_ma = tsmc_sheet.range(f"D{i}").value
    # 截取當天收盤價
    price_today = tsmc_sheet.range(f"B{i}").value

    # 若 5日 > 10日，而且我有足夠買入以今日收盤價計價的 1000 股的現金，就買入 1000 股（在 E 欄顯示 1000）
    if (short_term_ma > long_term_ma) and (tsmc_sheet.range(f"H{i-1}").value > price_today * 1000):
        tsmc_sheet.range(f"E{i}").value = 1000
    else:
        # 若上述條件不符和，就買入 0 股，（在 E 欄顯示 0）
        tsmc_sheet.range(f"E{i}").value = 0
    # 若 10日 > 5日，而且昨天的持有股數大於 1000 股，就賣出 1000 股
    if (long_term_ma > short_term_ma) and (tsmc_sheet.range(f"G{i-1}").value >= 1000):
        tsmc_sheet.range(f"F{i}").value = 1000
    else:
        tsmc_sheet.range(f"F{i}").value = 0
    # 持有股數，算法是前一天的持有股數 + 今天的買入股數 - 今天的賣出股數
    tsmc_sheet.range(f"G{i}").value = tsmc_sheet.range(f"G{i-1}").value + tsmc_sheet.range(f"E{i}").value - tsmc_sheet.range(f"F{i}").value
    # 持有資金，算法是前一天的持有資金 + 今日收盤價 x (今天的賣出股數 - 今天的買入股數)
    tsmc_sheet.range(f"H{i}").value = tsmc_sheet.range(f"H{i-1}").value + price_today * (tsmc_sheet.range(f"F{i}").value - tsmc_sheet.range(f"E{i}").value)
    # 總資產則是持有股數 x 今日收盤價 + 今日持有資金
    tsmc_sheet.range(f"I{i}").value = tsmc_sheet.range(f"H{i}").value + tsmc_sheet.range(f"G{i}").value * price_today

# 計算并且將總收益顯示在 L20
tsmc_sheet.range("L20").value = tsmc_sheet.range(f"I{last_row}").value - tsmc_sheet.range("L18").value

# 回家作業

請搜尋並開啓 `2330_sma_back_test.xlsx` 範例檔案，並且計算出三日移動平均，接下來請根據以下策略進行回測：

- 若當日的收盤價是大於當日的移動平均，就買進股票
- 若當日的收盤價是小於當日的移動平均，就賣出股票

若卡關的同學，可以參考這篇文章：

Medium：[用 Python 與 Excel 打造簡易的回測系統（上）](https://medium.com/pyradise/%E7%94%A8-python-%E8%88%87-excel-%E6%89%93%E9%80%A0%E7%B0%A1%E6%98%93%E7%9A%84%E5%9B%9E%E6%B8%AC%E7%B3%BB%E7%B5%B1-857bfbb88867)